In [1]:
import mountaineer as mtr
import folium
import pandas as pd
import ee
ee.Initialize()

# Map with folium

In [30]:
# load 'world' dataset
world = mtr.load('world')

In [31]:
w = mtr.Dataset(world)

In [ ]:
# init a map instance
fmap = folium.Map()

# create a layer to hold several features
region = folium.FeatureGroup(name="World")

# iterate over rows of 
for idx in world.index:
    
    # init a GeoJson object from each geometry datum and add to world layer
    data = folium.GeoJson(
        data=world.loc[idx, "geometry"],
        tooltip=world.loc[idx, "Name"],
    ).add_to(region)   
    
# add layer to the map
fmap.add_child(region)

# set bounds (location and zoom) on map to center on this region
fmap.fit_bounds(region.get_bounds())

# add layer toggle option
fmap.add_child(folium.LayerControl())

# show map
fmap

In [ ]:
# Mountains in the U.S.

# init a map instance
fmap = folium.Map(tiles="Stamen Terrain")

# create a layer to hold several features
region = folium.FeatureGroup(name="U.S. Mountains")

# iterate over rows of 
for idx in world[world.Country == "United States of America"].index:
    
    # init a GeoJson object from each geometry datum and add to world layer
    data = folium.GeoJson(
        data=world.loc[idx, "geometry"],
        tooltip=world.loc[idx, "Name"],
    ).add_to(region)   
    

# add layer to the map
fmap.add_child(region)

# set bounds (location and zoom) on map to center on this region
fmap.fit_bounds(region.get_bounds())

# add layer toggle option
fmap.add_child(folium.LayerControl())

# show map
fmap

In [ ]:
# Mountains in China

# init a map instance
fmap = folium.Map(tiles="https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png",
                 attr="https://www.openstreetmap.org/copyright")

# create a layer to hold several features
region = folium.FeatureGroup(name="China")

# iterate over rows of 
for idx in world[world.Country == "China"].index:
    
    # init a GeoJson object from each geometry datum and add to world layer
    data = folium.GeoJson(
        data=world.loc[idx, "geometry"],
        tooltip=world.loc[idx, "Name"],
    ).add_to(region)   
    
# add layer to the map
fmap.add_child(region)

# set bounds (location and zoom) on map to center on this region
fmap.fit_bounds(region.get_bounds())

# add layer toggle option
fmap.add_child(folium.LayerControl())

# show map
fmap

# earth engine

In [ ]:
class EngineMap(folium.Map):
    """
    Subclass of folium Map for adding EE images. Here
    self is a folium.Map class instance, and we are adding 
    additional functions to this class type.
    """
    
    def __init__(self, location=None, zoom_start=None, **kwargs):
        # inherit from parent class
        super().__init__(location=location, zoom_start=zoom_start, **kwargs)
    
    
    def add_ee_vector(self, feature, **kwargs):
        """
        Add a google earth engine vector feature to a folium map.
        """
        # create vector feature
        feature = folium.GeoJson(
            data=feature.geometry().getInfo(),
            **kwargs,
        )
        
        # add vector to Map and return self to allow chaining
        self.add_child(feature)
        return self
    
    
    def add_ee_raster(self, image, vis_params={}, **kwargs):
        """
        Add a google earth engine raster layer to a folium map.
        """       
        # handle ImageCollections and Images
        if isinstance(image, ee.ImageCollection):
            ee_image = image.mosaic()

        # get the JSON instructions to show image tiles
        map_id_dict = image.getMapId(vis_params)
        
        # get url of the raster tiles
        tiles = map_id_dict['tile_fetcher'].url_format
        
        # create a folium raster layer and add to self
        raster = folium.raster_layers.TileLayer(
            tiles=tiles,
            attr=kwargs.get("attr", "TODO-attr"),
            name=kwargs.get("name", "test"),
            overlay=True,
            control=True,
        )
        
        # add raster to Map and return self to allow chaining
        self.add_child(raster)
        return self

In [ ]:
# create an ee vector object
region = ee.Feature(
    ee.Geometry.Polygon([
        [ -111.192006505813765, 41.668436084383927 ], [ -111.185124574048075, 41.537679380837119 ], [ -111.231642943020063, 41.544570991055082 ], [ -111.260196475090027, 41.30186596846039 ], [ -111.136510368515019, 41.183071273109476 ], [ -111.129388300123821, 41.140187778029713 ], [ -111.229879232190285, 40.979390702550688 ], [ -111.229879232190285, 40.937988081049355 ], [ -111.354562988895793, 40.937988081049355 ], [ -111.354562988895793, 40.895919210466161 ], [ -111.396441502598407, 40.895824032025928 ], [ -111.396441502598407, 40.813018789023033 ], [ -111.354087096694684, 40.813113967463266 ], [ -111.354087096694684, 40.729547296938506 ], [ -111.313160367394346, 40.72935694005804 ], [ -111.313160367394346, 40.646075804853979 ], [ -111.301326761473945, 40.608151784417771 ], [ -111.313160367394346, 40.562604312769452 ], [ -111.313160367394346, 40.521201691268061 ], [ -111.396441502598407, 40.521201691268061 ], [ -111.396441502598407, 40.479513534445857 ], [ -111.312684475193237, 40.479513534445857 ], [ -111.312684475193237, 40.396232399241796 ], [ -111.271281853691733, 40.396232399241796 ], [ -111.271281853691733, 40.35435388553924 ], [ -111.229879232190285, 40.35435388553924 ], [ -111.229879232190285, 40.270596858133956 ], [ -111.188000718487672, 40.270596858133956 ], [ -111.090458837176584, 40.173439448173838 ], [ -111.271290201835086, 40.104386044540774 ], [ -111.312632261040505, 40.104403049468374 ], [ -111.312632261040505, 40.020646022063318 ], [ -111.312632261040505, 39.937840779060252 ], [ -111.395913396244566, 39.937840779060252 ], [ -111.395999281313948, 39.813107495268241 ], [ -111.395999281313948, 39.728950668906577 ], [ -111.437859671628701, 39.728950668906577 ], [ -111.437859671628701, 39.575898616818733 ], [ -111.599777682866375, 39.542200661180914 ], [ -111.669681313218291, 39.274236744832024 ], [ -111.803663271392736, 39.105302971481592 ], [ -111.896868111861863, 39.250935534714699 ], [ -111.960946439684392, 39.454821123241061 ], [ -111.86191629668599, 39.775212762353817 ], [ -111.84444038909794, 39.973273048350791 ], [ -111.768711456216749, 40.031526073644045 ], [ -111.634565006673881, 40.128395115751744 ], [ -111.713087219866281, 40.306854691189017 ], [ -111.803663271392736, 40.358908075792044 ], [ -111.912961944356027, 40.421068819468871 ], [ -111.919647408617777, 40.494608926348519 ], [ -111.920100327373518, 40.499591032661272 ], [ -111.919618470344062, 40.499872115928554 ], [ -111.852831684971022, 40.538831074062728 ], [ -111.869992844358421, 40.598695583553479 ], [ -111.81044804669807, 40.725228278581881 ], [ -111.955792242460973, 40.777987970343418 ], [ -111.970069008495955, 40.856510183535818 ], [ -111.67291259765625, 41.018695831298885 ], [ -112.040962219238224, 41.335079193115234 ], [ -111.862907409667912, 41.564250946044979 ], [ -111.885360519417588, 41.699710934235895 ], [ -111.862993263233591, 41.763084826757165 ], [ -111.827301348146136, 42.048620147456802 ], [ -111.848716497198609, 42.350641960875009 ], [ -111.739665610264183, 42.564199947788325 ], [ -111.722109035982214, 42.670388119427741 ], [ -111.672876462431589, 42.745084473599604 ], [ -111.598686656305688, 42.70580751741528 ], [ -111.588127077807769, 42.629611001722367 ], [ -111.465795724691986, 42.565532673899838 ], [ -111.32598846398821, 42.408249505608126 ], [ -111.32598846398821, 42.02377953867267 ], [ -111.349289674105364, 41.843195160263747 ], [ -111.192006505813765, 41.668436084383927 ],
    ])
)

# create a map
emap = EngineMap()

# add ee vector feature
emap.add_ee_vector(region)

# focus location/bounds of the map on its child features
emap.fit_bounds(emap.get_bounds(), max_zoom=5)
emap

In [ ]:
# create an ee vector object
region = ee.Feature(
    ee.Geometry.Polygon([
        [ -120.905959467999196, 45.042710296407165 ], [ -120.890142816664422, 44.876919958164876 ], [ -120.933912166270034, 44.758742714229811 ], [ -121.113091946459917, 44.651460059314672 ], [ -121.327965146051724, 44.551393085057441 ], [ -121.575523055989493, 44.318597668737368 ], [ -121.544012341290227, 44.288552568675414 ], [ -121.576607440808687, 44.242919429349513 ], [ -121.576607440808687, 44.195330209232907 ], [ -121.505223610633777, 44.153689641630876 ], [ -121.505223610633777, 44.046613896368513 ], [ -121.505223610633777, 43.987127371222755 ], [ -121.523069568177505, 43.975230066193603 ], [ -121.624196660925293, 43.90979488853327 ], [ -121.671785881041899, 43.868154320931239 ], [ -121.671785881041899, 43.772975880698027 ], [ -121.632418400178892, 43.724860070754573 ], [ -121.654833117474823, 43.707549695020987 ], [ -121.621000259594041, 43.613694184772044 ], [ -121.695580491100202, 43.523132475085845 ], [ -121.826450846420869, 43.422005382338057 ], [ -121.850245456479172, 43.368467509706875 ], [ -121.83834815145002, 43.261391764444511 ], [ -121.833067703738436, 43.256551354042244 ], [ -121.921629286654081, 43.195956586784177 ], [ -121.957321201741536, 43.112675451580117 ], [ -121.957321201741536, 42.993702401288601 ], [ -121.90973198162493, 42.916369918599116 ], [ -121.896644946092863, 42.90328288306705 ], [ -121.939475244197808, 42.82714013088048 ], [ -121.963269854256112, 42.773602258249298 ], [ -122.034653684431021, 42.731961690647267 ], [ -122.0643969470039, 42.648680555443207 ], [ -122.082242904547627, 42.488066937549661 ], [ -122.040602336945597, 42.434529064918479 ], [ -121.951684710007044, 42.403601194679027 ], [ -121.953129220217136, 42.396017516075915 ], [ -121.948509506210257, 42.388625973665057 ], [ -121.969218506770687, 42.333401972170691 ], [ -121.987064464314415, 42.220377574393751 ], [ -121.839037262814429, 42.198447618616001 ], [ -121.891886024081202, 41.976482821296145 ], [ -121.707477796129353, 41.869407076033781 ], [ -121.511172263148353, 41.762331330771417 ], [ -121.427891127944292, 41.714742110654811 ], [ -121.37435325531311, 41.619563670421599 ], [ -121.386250560342262, 41.506539272644659 ], [ -121.415993822915141, 41.423258137440598 ], [ -121.588504745837838, 41.458950052528053 ], [ -121.731272406187657, 41.47679601007178 ], [ -121.778861626304263, 41.387566222353144 ], [ -121.802533820275073, 41.269205252498978 ], [ -122.062727076398232, 41.27071882453896 ], [ -122.270921745278997, 41.27071882453896 ], [ -122.314719458640013, 41.27071882453896 ], [ -122.362826581544198, 41.389094778651554 ], [ -122.343983615188961, 41.488693315100932 ], [ -122.349932267703537, 41.58982040784872 ], [ -122.349932267703537, 41.720690763169387 ], [ -122.427264750393022, 41.809920550888023 ], [ -122.479335584273826, 41.816429405123131 ], [ -122.479335584273826, 41.854355041315955 ], [ -122.562854663062183, 41.853468318328112 ], [ -122.687325182781933, 41.854445411089443 ], [ -122.767194489479948, 41.853468318328112 ], [ -122.799158261210096, 41.854514992167424 ], [ -122.853953298461761, 41.854514992167424 ], [ -122.978954477192076, 41.854039099966201 ], [ -123.143176613372759, 41.855060286293565 ], [ -123.143176613372759, 41.940024454891841 ], [ -123.227792568165569, 41.940024454891841 ], [ -123.227792568165569, 41.980765470162396 ], [ -123.187298204340721, 41.980685671165304 ], [ -123.187238717815603, 42.021138916625773 ], [ -123.146140994860048, 42.021138916625773 ], [ -123.146140994860048, 42.062581998597807 ], [ -123.104352553871593, 42.062581998597807 ], [ -123.104352553871593, 42.104025080569841 ], [ -123.062909471899559, 42.104025080569841 ], [ -123.062909471899559, 42.146158880574717 ], [ -123.062909471899559, 42.270488126490704 ], [ -123.229372517820366, 42.270488126490704 ], [ -123.229372517820366, 42.312967285511945 ], [ -123.354737840785674, 42.312967285511945 ], [ -123.354737840785674, 42.354065008467444 ], [ -123.396180922757708, 42.354065008467444 ], [ -123.396180922757708, 42.395853449455899 ], [ -123.437855066410918, 42.604590939250897 ], [ -123.437855066410918, 42.979058615043471 ], [ -123.406443947984087, 43.19968232730082 ], [ -123.346715719464498, 43.410722068070072 ], [ -123.176794967229569, 43.439851339881784 ], [ -123.306229062164675, 43.531533823794121 ], [ -123.266586884032733, 43.641021744348848 ], [ -123.141103052142114, 43.654002830406512 ], [ -123.129205747112962, 43.689694745493966 ], [ -123.235425250128458, 43.755449675932141 ], [ -123.235590902797128, 43.770524068781299 ], [ -123.039975959394326, 43.820565100814633 ], [ -123.058871679146421, 43.905595839699458 ], [ -122.980489434248568, 43.933589498591573 ], [ -122.998335391792295, 44.034716591339361 ], [ -123.03402730687975, 44.100151768999694 ], [ -123.051873264423477, 44.21912481929121 ], [ -123.022130001850599, 44.349995174611877 ], [ -123.010232696821447, 44.45707091987407 ], [ -122.996494142204483, 44.544081765781755 ], [ -122.92100290910281, 44.534403402563555 ], [ -122.903156951559083, 44.474916877417797 ], [ -122.855567731442477, 44.361892479641028 ], [ -122.754440638694689, 44.37378978467018 ], [ -122.694954113548931, 44.439224962330513 ], [ -122.689005461034355, 44.510608792505252 ], [ -122.754440638694689, 44.564146665136434 ], [ -122.819875816355022, 44.653376452855071 ], [ -122.811768963690326, 44.72633812683722 ], [ -122.677108156005204, 44.760452198117434 ], [ -122.587878368286567, 44.867527943379798 ], [ -122.647364893432325, 44.962706383613011 ], [ -122.564083758228264, 45.093576738933677 ], [ -122.361829572732688, 45.23039574676892 ], [ -122.225010564897445, 45.283933619400102 ], [ -122.183369997295415, 45.367214754604163 ], [ -122.296394395072355, 45.492136457410254 ], [ -122.409418792849294, 45.533777025012284 ], [ -122.41774040511001, 45.567063474055601 ], [ -122.361829572732688, 45.581366245128891 ], [ -122.4570080129659, 45.706287947934982 ], [ -122.474853970509628, 45.795517735653618 ], [ -122.480802623024204, 45.890696175886831 ], [ -122.558135105713689, 45.902593480915982 ], [ -122.653313545946901, 45.884747523372255 ], [ -122.748491986180113, 45.896644828401406 ], [ -122.843670426413325, 45.950182701032588 ], [ -122.867465036471629, 46.027515183722073 ], [ -122.923565705078431, 46.013490016570472 ], [ -122.982634432732993, 46.095260234803959 ], [ -122.918909623868444, 46.29197594912489 ], [ -122.778235248752992, 46.259512631790528 ], [ -122.712800071092659, 46.283307241848831 ], [ -122.659262198461477, 46.336845114480013 ], [ -122.61167297834487, 46.396331639625771 ], [ -122.629518935888598, 46.515304689917286 ], [ -122.57003241074284, 46.58073986757762 ], [ -122.700902766063507, 46.646175045237953 ], [ -122.72469737612181, 46.729456180442014 ], [ -122.754440638694689, 46.800840010616923 ], [ -122.74271329516597, 46.863385842770185 ], [ -122.629518935888598, 46.860326535762681 ], [ -122.52839184314081, 46.860326535762681 ], [ -122.361829572732688, 46.907915755879287 ], [ -122.165524039751688, 46.979299586054196 ], [ -122.070345599518475, 47.056632068743681 ], [ -121.987064464314415, 47.1993997290935 ], [ -121.957321201741536, 47.371910652016197 ], [ -122.129832124664233, 47.478986397278561 ], [ -122.134413891037752, 47.588948790246491 ], [ -121.969218506770687, 47.597959447570076 ], [ -121.933526591683233, 47.651497320201258 ], [ -121.766964321275111, 47.675291930259561 ], [ -121.957321201741536, 47.740727107919895 ], [ -122.010859074372718, 47.812110938094804 ], [ -122.010859074372718, 47.889793341520544 ], [ -121.945423896712384, 47.901340725813441 ], [ -121.808604888877142, 47.85970015821141 ], [ -121.802656236362566, 47.925135335871744 ], [ -121.885937371566627, 47.996519166046653 ], [ -121.963269854256112, 48.038159733648683 ], [ -122.034653684431021, 48.08574895376529 ], [ -122.088191557062203, 48.174978741483926 ], [ -122.141729429693385, 48.24041391914426 ], [ -122.236907869926597, 48.299900444290017 ], [ -122.296549302537358, 48.290724839273025 ], [ -122.382842530155131, 48.364408922701273 ], [ -122.290445742557779, 48.43671945212526 ], [ -122.248805174955748, 48.520000587329321 ], [ -122.278548437528627, 48.555692502416775 ], [ -122.379675530276415, 48.555692502416775 ], [ -122.486751275538779, 48.621127680077109 ], [ -122.512186893187334, 48.688955993806644 ], [ -122.409418792849294, 48.7460493828832 ], [ -122.338034962674385, 48.882868390718443 ], [ -122.319699833043103, 48.953153054305005 ], [ -122.203340413459387, 49.031919430638482 ], [ -122.141785191020688, 49.057390557164865 ], [ -122.061126623687414, 49.116823185726332 ], [ -121.895592420644732, 49.191113545578446 ], [ -121.77083253783303, 49.229363401661089 ], [ -121.687445838142651, 49.229363401661089 ], [ -121.687445838142651, 49.312487051825599 ], [ -121.604059138452214, 49.312487051825599 ], [ -121.604190663215206, 49.354048876907882 ], [ -121.520803963524941, 49.354048876907882 ], [ -121.354162088907174, 49.354048876907882 ], [ -121.354241003764912, 49.396031581231398 ], [ -121.236789390478691, 49.395978971326315 ], [ -121.148667799323277, 49.395978971326315 ], [ -121.02108877929237, 49.395978971326315 ], [ -121.02108877929237, 49.437540796408484 ], [ -120.937702079602047, 49.437540796408484 ], [ -120.937702079602047, 49.56248932118109 ], [ -120.854315379911611, 49.56248932118109 ], [ -120.687673505293958, 49.56248932118109 ], [ -120.646111680211618, 49.56248932118109 ], [ -120.646111680211618, 49.520927496098921 ], [ -120.521031630676191, 49.520927496098921 ], [ -120.520900105913313, 49.479102621490767 ], [ -120.312564881450328, 49.479102621490767 ], [ -120.270871531605167, 49.479102621490767 ], [ -120.270871531605167, 49.437540796408484 ], [ -120.104098132224522, 49.437540796408484 ], [ -120.104098132224522, 49.395978971326315 ], [ -120.063851554771418, 49.395978971326315 ], [ -120.063851554771418, 49.353891047192405 ], [ -120.020974482060069, 49.353891047192405 ], [ -120.020974482060069, 49.262349812201023 ], [ -120.020974482060069, 49.187538527052993 ], [ -119.906766407758084, 49.187528127790017 ], [ -119.729139643672852, 49.187528127790017 ], [ -119.729139643672852, 49.104246992585956 ], [ -119.729139643672852, 49.020727911281369 ], [ -119.687855995221696, 49.020727911281369 ], [ -119.604336913916995, 49.020727911281369 ], [ -119.604336913916995, 49.062606424983926 ], [ -119.479415211110904, 49.062606424983926 ], [ -119.437584286628407, 49.062558835763866 ], [ -119.437584286628407, 49.023535675268192 ], [ -119.437584286628407, 48.937161240756609 ], [ -119.437584286628407, 48.854118051653018 ], [ -119.39618166512696, 48.854118051653018 ], [ -119.39618166512696, 48.771074862549654 ], [ -119.43782223272899, 48.771074862549654 ], [ -119.437703259678756, 48.645915213642866 ], [ -119.479105881180203, 48.645915213642866 ], [ -119.479105881180203, 48.479352943234744 ], [ -119.479201059620436, 48.354193294328184 ], [ -119.562482194824497, 48.354193294328184 ], [ -119.562482194824497, 48.312790672826623 ], [ -119.606621196482592, 48.312790672826623 ], [ -119.687522870680823, 48.312790672826623 ], [ -119.687522870680823, 48.187631023920062 ], [ -119.687760816781349, 48.062471375013445 ], [ -119.854442060239819, 48.062471375013445 ], [ -119.896131683606711, 48.06241991464185 ], [ -119.896131683606711, 47.938185842557857 ], [ -119.896131683606711, 47.854126967914056 ], [ -119.896131683606711, 47.770892199884258 ], [ -120.020983835651293, 47.770892199884258 ], [ -120.021101690643491, 47.645696205230422 ], [ -120.115879243424558, 47.645696205230422 ], [ -120.187459922307539, 47.645803965929815 ], [ -120.229148079129686, 47.645803965929815 ], [ -120.229148079129686, 47.604306165988078 ], [ -120.229148079129686, 47.562237295405055 ], [ -120.27093141439201, 47.562237295405055 ], [ -120.27093141439201, 47.520929852343784 ], [ -120.31261957121427, 47.520929852343784 ], [ -120.31261957121427, 47.479241695521694 ], [ -120.312559832123156, 47.437457598632193 ], [ -120.27094934090826, 47.437457598632193 ], [ -120.27094934090826, 47.39584710741741 ], [ -120.229338849693306, 47.39584710741741 ], [ -120.229338849693306, 47.354236616202456 ], [ -120.145891723289537, 47.354236616202456 ], [ -120.062595359535067, 47.354236616202456 ], [ -120.062595359535067, 47.187643888693515 ], [ -120.037710135174791, 47.187643888693515 ], [ -120.113238922223047, 47.128015898918591 ], [ -120.333339065262351, 47.0030941961125 ], [ -120.333339065262351, 46.89006979833556 ], [ -120.357133675320654, 46.788942705587772 ], [ -120.347785792797765, 46.774495978052414 ], [ -120.452312115553866, 46.669969655296256 ], [ -120.499901335670472, 46.616431782665074 ], [ -120.493952683155896, 46.31305050442171 ], [ -120.648617648534866, 46.205974759159346 ], [ -120.474076559552827, 46.133881700666734 ], [ -120.464209420583018, 45.985874616120043 ], [ -120.428517505495563, 45.896644828401406 ], [ -119.966183423339658, 45.781061307862558 ], [ -119.992871149750158, 45.756683885236782 ], [ -120.439911391462545, 45.675615091703833 ], [ -120.482629378144736, 45.609164890198258 ], [ -120.60102842841826, 45.617058160216345 ], [ -120.821128571457564, 45.623006812730921 ], [ -120.797333961399261, 45.509982414953981 ], [ -120.815179918942988, 45.38506071214789 ], [ -120.934152969234503, 45.283933619400102 ], [ -120.870045909687008, 45.098735447374168 ], [ -120.905959467999196, 45.042710296407165 ]
    ])
)

# create a map
emap = EngineMap()

# add ee vector feature
emap.add_ee_vector(region)

# focus location/bounds of the map on its child features
emap.fit_bounds(emap.get_bounds(), max_zoom=5)
emap

In [ ]:
# load a raster Image dataset from ee
raster = ee.Image('CGIAR/SRTM90_V4')

# create a map
emap = EngineMap(location=(45, -100), zoom_start=4)

# add raster to map
emap.add_ee_raster(raster, vis_params={'min': 0, 'max': 3000})

# draw map
emap

In [ ]:
# load a raster Image dataset from ee
raster = ee.Image('CGIAR/SRTM90_V4')

# create a map
emap = EngineMap(location=(45, -120), zoom_start=5)

# add raster to map
emap.add_ee_raster(raster, vis_params={'min': 0, 'max': 3000})

# add ee vector feature
emap.add_ee_vector(region)

# draw map
emap

In [ ]:
# load a raster Image dataset from ee
raster = ee.Image('CGIAR/SRTM90_V4')

# create a map
emap = EngineMap([45, -120], zoom_start=5)

# clip raster to polygon region of interest
clipped = raster.clip(region)

# add raster to map
emap.add_ee_raster(clipped, vis_params={'min': 0, 'max': 3000})

# show it
emap

# change markers to circle markers using style_function?

In [ ]:
def style_function(feature):
    employed = employed_series.get(int(feature["id"][-5:]), None)
    return {
        "fillOpacity": 0.5,
        "weight": 0,
        "fillColor": "#black" if employed is None else colorscale(employed),
    }


m = folium.Map(location=[48, -102], tiles="cartodbpositron", zoom_start=3)

folium.TopoJson(
    json.loads(requests.get(county_geo).text),
    "objects.us_counties_20m",
    style_function=style_function,
).add_to(m)

# fix code to convert json into geojson

In [ ]:
class write_GeoJSON:
    """
    Writes GBIF output to a GeoJSON format.
    """

    def __init__(self, data, name = "test", workdir = "."):
        self.data = data.reset_index()
        self.name = name
        self.workdir = workdir
        self.json_file = (
            os.path.join(self.workdir, self.name + ".json")
            .replace(" ", "_")
        )
        self.points = list(zip(data.longitude, data.latitude))
        self.feature_collection = geojson.FeatureCollection(
            features = [],
            properties = {"name": name},
        )

        # Attribute to store geographic range.
        self.georange = None

        # Run internal functions.
        self._add_points()
        self.write()

    @property
    def center(self):
        """
        Get the center of the geographic range.
        """
        return self.georange.centroid.xy[0][0], self.georange.centroid.xy[1][0]


    def _add_points(self):
        """
        Store observed point occurrences.
        """

        for idx in self.data.index:

            # Format from GBIF metadata.
            key = self.data.loc[idx, "key"]
            uri = f"https://www.gbif.org/occurrence/{key}"
            longitude = self.data.loc[idx, "longitude"]
            latitude = self.data.loc[idx, "latitude"]
            point = (longitude, latitude)
            geometry = geojson.Point(coordinates = point)

            # Write as a feature.
            feature = geojson.Feature(
                geometry = geometry, 
                properties = {
                    "type": "occurrence",
                    "record": f"<a href={uri} target='_blank'>{uri}</a>",
                    }
            )

            # Append to feature collection.
            self.feature_collection['features'].append(feature)
    

    def write(self):
        """
        Writes feature collection to GeoJSON file.
        """

        # If the provided workdir doesn't exist, make it.
        if not os.path.exists(self.workdir):
            os.makedirs(self.workdir)

        # Write feature collection to GeoJSON file.
        with open(self.json_file, 'w') as outf:
            outf.write(geojson.dumps(self.feature_collection, indent = 4))
        logger.info(f"wrote data to {self.json_file}")

In [ ]:
import os
import geojson
write_GeoJSON(data)